In [1]:
import scipy.sparse
import scipy.linalg
import numpy as np
import mkprop
from exlaser import doublewellproblem as prob
import matplotlib.pyplot as plt

n=2000
L=5
Hamiltonian = prob(n,L)

x, nrm, inr = Hamiltonian.getprop()
#tend = 5.0
tend = 0.05
u = Hamiltonian.getinitialstate()

mv, dmv = Hamiltonian.setupHamiltonian(0)

In [2]:
np.set_printoptions(linewidth=180)
m=40
tol=1e-4

In [3]:
y0=u
amat=[1]
cmat=[[1/2]]
tinit = 1e-4

tnow = 0
dt = min(tinit,tend-tnow)
nexps=1
while (tnow<tend):
    mv, dmv = Hamiltonian.setupHamiltonian(tnow+0.5*dt)
    y1, errest, tlist, mlist = mkprop.expimv_pKry(mv,y0,t=dt,m=m,ktype=2,reo=1,inr=inr)
    errst=0
    if errest<dt*tol:
        y0=y1
        tnow += dt
    dt = min(tinit,tend-tnow)
#yx=y1

In [4]:
tol=1e-4
y0=u

c = 3**0.5/6
cmat = [0.5-c, 0.5+c]
amat=[[0.25+c,0.25-c],[0.25-c,0.25+c]]
jexps=len(cmat)
tinit = 1e-4

p=2
tnow = 0
dt = min(tinit,tend-tnow)
nexps=1
sig = 1j
tlist=[]
dtlist=[]
errmlist=[]
spstep=0.9
spstepfail=0.7
errpartm=0.99
tolkry=(1-errpartm)*tol
mv0, _ = Hamiltonian.setupHamiltonian(tnow)
ye0 = mv0(y0)
while (tnow<tend):
    y0sub=y0
    chat = -0.5
    
    know=0
    anow = amat[know]
    mv, dmv = Hamiltonian.setupHamiltonianCFM(anow,cmat,chat,tnow,dt)
    y1sub, errestkry, tkrylist, mlist = mkprop.expimv_pKry(mv,y0sub,tol=tolkry,t=dt,m=m,ktype=2,reo=0)

    know=1
    anow = amat[know]
    mv, dmv = Hamiltonian.setupHamiltonianCFM(anow,cmat,chat,tnow,dt)
    y1, errestkry, tkrylist, mlist = mkprop.expimv_pKry(mv,y1sub,tol=tolkry,t=dt,m=m,ktype=2,reo=0)

    y0 = y1
    tnow += dt
    dtlist.append(dt)
    tlist.append(tnow)


In [5]:
print(nrm(y1-yx)/tend)

NameError: name 'yx' is not defined

In [ ]:
yx,_,_ = CFMKprop.midpoint(u,0,tend,1e-4,Hamiltonian,ktol=1e-4,ktype=2,reo=0,inr=inr)

In [ ]:
tol=1e-5
y0=u

c = 3**0.5/6
cmat = [0.5-c, 0.5+c]
amat=[[0.25+c,0.25-c],[0.25-c,0.25+c]]
chat = -0.5
jexps=len(cmat)
tinit = 0.0001

p=4
tnow = 0
dt = min(tinit,tend-tnow)
nexps=1
sig = 1j
tlist=[]
dtlist=[]
errmlist=[]
spstep=0.9
spstepfail=0.7
errpartm=0.99
tolkry=(1-errpartm)*tol
mv0, _ = Hamiltonian.setupHamiltonian(tnow)
ye0 = mv0(y0)
ye0 = ye0+0j
while (tnow<tend):
    y0sub=y0
    e0sub=-0.5*ye0

    know=0
    anow = amat[know]
    mv, dmv = Hamiltonian.setupHamiltonianCFM(anow,cmat,chat,tnow,dt)

    dHpsi = dmv(y0sub)
    Hpsi = mv(y0sub)
    HdHpsi = mv(dHpsi)
    dHHpsi = dmv(Hpsi)
    Gam11 = Hpsi + dt/2*dHpsi - sig*dt**2*1/12*(HdHpsi-dHHpsi)
    e0sub += Gam11

    y1sub, errestkry, tkrylist, mlist = mkprop.expimv_pKry(mv,y0sub,tol=tolkry,t=dt,m=m,ktype=2,reo=0)
    e1sub, errestkry, tkrylist, mlist = mkprop.expimv_pKry(mv,e0sub,tol=tolkry,t=dt,m=m,ktype=2,reo=0)

    dHpsi = dmv(y1sub)
    Hpsi = mv(y1sub)
    HdHpsi = mv(dHpsi)
    dHHpsi = dmv(Hpsi)
    Gam12 = dt/2*dHpsi + sig*dt**2*1/12*(HdHpsi-dHHpsi)
    e1sub += Gam12

    know=1
    anow = amat[know]
    mv, dmv = Hamiltonian.setupHamiltonianCFM(anow,cmat,chat,tnow,dt)

    dHpsi = dmv(y1sub)
    Hpsi = mv(y1sub)
    HdHpsi = mv(dHpsi)
    dHHpsi = dmv(Hpsi)
    Gam21 = dt/2*dHpsi - sig*dt**2*1/12*(HdHpsi-dHHpsi)
    e1sub += Gam21

    y2sub, errestkry, tkrylist, mlist = mkprop.expimv_pKry(mv,y1sub,tol=tolkry,t=dt,m=m,ktype=2,reo=0)
    e2sub, errestkry, tkrylist, mlist = mkprop.expimv_pKry(mv,e1sub,tol=tolkry,t=dt,m=m,ktype=2,reo=0)

    dHpsi = dmv(y2sub)
    Hpsi = mv(y2sub)
    HdHpsi = mv(dHpsi)
    dHHpsi = dmv(Hpsi)
    Gam22 = Hpsi + dt/2*dHpsi + sig*dt**2*1/12*(HdHpsi-dHHpsi)
    e2sub += Gam22

    # computer error estimate

    y1 = y2sub
    mv2, _ = Hamiltonian.setupHamiltonian(tnow+dt)
    ye1 = mv2(y1)
    def1 = e2sub-0.5*ye1
    errestmag = dt/(p+1)*nrm(def1)

    # test error estimate
    if errestkry+errestmag<dt*tol:
        y0 = y1
        ye0 = ye1
        tnow += dt
        dtlist.append(dt)
        tlist.append(tnow)
        errmlist.append(errestmag)
        dtnew = spstep * (errpartm * dt * tol/errestmag)**(1.0/p) * dt
        print("try dt=%.3e, errest=%.2e, newdt=%.3e - used" % (dt,(errestkry+errestmag)/dt,dtnew))
        dt = min(dtnew,tend-tnow)
    else:
        print("try dt=%.3e, errest=%.2e, failed" % (dt,(errestkry+errestmag)/dt))
        dtnew = spstepfail * (errpartm * dt * tol/errestmag)**(1.0/p) * dt
        dt = min(dtnew,tend-tnow)
print("done")

In [ ]:
print(nrm(y1-yx)/tend)

In [ ]:
tol=1e-6
y0=u
amat=[[1]]
cmat=[1/2]
tinit = 0.0001

p=2
tnow = 0
dt = min(tinit,tend-tnow)
nexps=1
sig = 1j
tlist=[]
dtlist=[]
errmlist=[]
spstep=0.9
spstepfail=0.7
errpartm=0.99
tolkry=(1-errpartm)*tol
while (tnow<tend):
    know=0
    anow = amat[know]
    chat = 0
    mv, dmv = Hamiltonian.setupHamiltonianCFM(anow,cmat,chat,tnow,dt)
    y1, errestkry, tkrylist, mlist = mkprop.expimv_pKry(mv,y0,tol=tolkry,t=dt,m=m,ktype=2,reo=0)

    # computer error estimate
    dHpsi = dmv(y1)
    HdHpsi = mv(dHpsi)
    Hpsi = mv(y1)
    dHHpsi = dmv(Hpsi)
    defp1 = Hpsi + dt*dHpsi + sig*dt**2/2*(HdHpsi-dHHpsi)
    
    mv2, _ = Hamiltonian.setupHamiltonian(tnow+dt)
    def1 = defp1-mv2(y1)
    errestmag = dt/(p+1)*nrm(def1)

    # test error estimate
    if errestkry+errestmag<dt*tol:
        y0 = y1
        tnow += dt
        dtlist.append(dt)
        tlist.append(tnow)
        errmlist.append(errestmag)
        dtnew = spstep * (errpartm * dt * tol/errestmag)**(1.0/p) * dt
        print("try dt=%.3e, errest=%.2e, newdt=%.3e - used" % (dt,(errestkry+errestmag)/dt,dtnew))
        dt = min(dtnew,tend-tnow)
    else:
        print("try dt=%.3e, errest=%.2e, failed" % (dt,(errestkry+errestmag)/dt))
        dtnew = spstepfail * (errpartm * dt * tol/errestmag)**(1.0/p) * dt
        dt = min(dtnew,tend-tnow)


In [ ]:
print(nrm(y1-yx)/tend)

In [ ]:
tol=1e-4
y0=u
tinit = 0.0001

p=2
tnow = 0
dt = min(tinit,tend-tnow)
nexps=1
sig = 1j
tlist=[]
dtlist=[]
errmlist=[]
spstep=0.9
spstepfail=0.7
errpartm=0.99
tolkry=(1-errpartm)*tol
while (tnow<tend):
    mv, dmv = Hamiltonian.setupHamiltonian(tnow+0.5*dt)
    y1, errestkry, tkrylist, mlist = mkprop.expimv_pKry(mv,y0,tol=tolkry,t=dt,m=m,ktype=2,reo=0)

    # computer error estimate
    dHpsi = dmv(y1)
    HdHpsi = mv(dHpsi)
    Hpsi = mv(y1)
    dHHpsi = dmv(Hpsi)
    defp1 = Hpsi + 0.5*dt*dHpsi + 0.5*sig*dt**2/2*(HdHpsi-dHHpsi)
    
    mv2, _ = Hamiltonian.setupHamiltonian(tnow+dt)
    def1 = defp1-mv2(y1)
    errestmag = dt/(p+1)*nrm(def1)

    # test error estimate
    if errestkry+errestmag<dt*tol:
        y0 = y1
        tnow += dt
        dtlist.append(dt)
        tlist.append(tnow)
        errmlist.append(errestmag)
        dtnew = spstep * (errpartm * dt * tol/errestmag)**(1.0/p) * dt
        print("try dt=%.3e, errest=%.2e, newdt=%.3e - used" % (dt,(errestkry+errestmag)/dt,dtnew))
        dt = min(dtnew,tend-tnow)
    else:
        print("try dt=%.3e, errest=%.2e, failed" % (dt,(errestkry+errestmag)/dt))
        dtnew = spstepfail * (errpartm * dt * tol/errestmag)**(1.0/p) * dt
        dt = min(dtnew,tend-tnow)



In [ ]:
plt.semilogy(tlist,dtlist)

In [ ]:
print(nrm(y1-yx)/tend)

In [ ]:
tol=1e-4
y0=u

c = 3**0.5/6
cmat = [0.5-c, 0.5+c]
amat=[[0.25+c,0.25-c],[0.25-c,0.25+c]]
chat = -0.5
jexps=len(cmat)
tinit = 0.0001

p=4
tnow = 0.5234
dt = min(tinit,tend-tnow)
nexps=1
sig = 1j
tlist=[]
dtlist=[]
errmlist=[]
spstep=0.9
spstepfail=0.7
errpartm=0.99
tolkry=(1-errpartm)*tol
mv0, _ = Hamiltonian.setupHamiltonian(tnow)
ye0 = mv0(y0)
ye0 = ye0+0j

dts = np.geomspace(5e-1,1e-3,20)
errlist = np.zeros(dts.size)
errestlist1 = np.zeros(dts.size)
errestlist2 = np.zeros(dts.size)
for dt, ij in zip(dts,range(len(dts))):
    
    y0sub=y0
    e0sub=-0.5*ye0

    know=0
    anow = amat[know]
    mv, dmv = Hamiltonian.setupHamiltonianCFM(anow,cmat,chat,tnow,dt)

    dHpsi = dmv(y0sub)
    Hpsi = mv(y0sub)
    HdHpsi = mv(dHpsi)
    dHHpsi = dmv(Hpsi)
    Gam11 = Hpsi + dt/2*dHpsi - sig*dt**2*1/12*(HdHpsi-dHHpsi)
    e0sub += Gam11

    y1sub, errestkry, tkrylist, mlist = mkprop.expimv_pKry(mv,y0sub,tol=tolkry,t=dt,m=m,ktype=2,reo=0)
    e1sub, errestkry, tkrylist, mlist = mkprop.expimv_pKry(mv,e0sub,tol=tolkry,t=dt,m=m,ktype=2,reo=0)

    dHpsi = dmv(y1sub)
    Hpsi = mv(y1sub)
    HdHpsi = mv(dHpsi)
    dHHpsi = dmv(Hpsi)
    Gam12 = dt/2*dHpsi + sig*dt**2*1/12*(HdHpsi-dHHpsi)
    e1sub += Gam12

    know=1
    anow = amat[know]
    mv, dmv = Hamiltonian.setupHamiltonianCFM(anow,cmat,chat,tnow,dt)

    dHpsi = dmv(y1sub)
    Hpsi = mv(y1sub)
    HdHpsi = mv(dHpsi)
    dHHpsi = dmv(Hpsi)
    Gam21 = dt/2*dHpsi - sig*dt**2*1/12*(HdHpsi-dHHpsi)
    e1sub += Gam21

    y2sub, errestkry, tkrylist, mlist = mkprop.expimv_pKry(mv,y1sub,tol=tolkry,t=dt,m=m,ktype=2,reo=0)
    e2sub, errestkry, tkrylist, mlist = mkprop.expimv_pKry(mv,e1sub,tol=tolkry,t=dt,m=m,ktype=2,reo=0)

    dHpsi = dmv(y2sub)
    Hpsi = mv(y2sub)
    HdHpsi = mv(dHpsi)
    dHHpsi = dmv(Hpsi)
    Gam22 = Hpsi + dt/2*dHpsi + sig*dt**2*1/12*(HdHpsi-dHHpsi)
    e2sub += Gam22

    # computer error estimate

    y1 = y2sub
    mv2, _ = Hamiltonian.setupHamiltonian(tnow+dt)
    ye1 = mv2(y1)
    def1 = e2sub-0.5*ye1
    errestmag = dt/(p+1)*nrm(def1)
    errestlist1[ij] = errestmag
    print(dt)
print("done")

In [ ]:
plt.loglog(dts, errestlist1,dts,1e1*dts**(p+1),':')

In [ ]:
c = 3**0.5/6
cmat = [0.5-c, 0.5+c]
amat=[[0.25+c,0.25-c],[0.25-c,0.25+c]]
parth1 = [1,0]
parth2 = [0,1]

In [ ]:
c = 15**0.5
cmat = [0.5-c/10, 0.5, 0.5+c/10]
#amat = [[0.302146842308616954258187683416,
#         -0.030742768872036394116279742324,
#         0.004851603407498684079562131338],
#        [-0.029220667938337860559972036973,
#         0.505929982188517232677003929089,
#         -0.029220667938337860559972036973],
#        [0.004851603407498684079562131337,
#         -0.030742768872036394116279742324,
#         0.302146842308616954258187683417]]

amat = [[37.0/240 + 10*c/3/87, -1.0/30, 37.0/240 - 10*c/3/87],
        [-11.0/360, 23.0/45, -11.0/360],
        [37.0/240 - 10*c/3/87, -1.0/30, 37.0/240 + 10*c/3/87]]

sa=0.5/sum(amat[0])
parth1 = [sa, 0.5, 1-sa]
parth2 = [1-sa, 0.5, sa]

In [ ]:
c=15**0.5
cmat = [0.5-c/10, 0.5, 0.5+c/10]
amat = [[0.79124225942889763, -0.080400755305553218, 0.012765293626634554],
        [-0.48931475164583259, 0.054170980027798808,-0.012069823881924156],
        [-0.029025638294289255, 0.50138457552775674,-0.025145341733509552],
        [0.0048759082890019896,-0.030710355805557892,0.30222764976657693]]

In [ ]:
c = 3**0.5/6
cmat2 = [0.5-c, 0.5+c]
amat2 = [[0.25+c,0.25-c],[0.25-c,0.25+c]]
jexps=len(amat2)

chat = -0.5
jexps=len(amat)
p=4

tnow = 0.522345
sig = 1j
tolkry=1e-8

y0=u
mv0, _ = Hamiltonian.setupHamiltonian(tnow)
ye0 = mv0(y0)
ye0 = ye0+0j

dts = np.geomspace(1e-1,5e-3,8)
errlist = np.zeros(dts.size)
errestlist1 = np.zeros(dts.size)
errestlist2 = np.zeros(dts.size)
for dt, ij in zip(dts,range(len(dts))):
    print(ij)
    y0sub=u
    e0sub=-0.5*ye0
    for know in range(jexps):
        anow = amat[know]
        mv, dmv = Hamiltonian.setupHamiltonianCFM(anow,cmat,chat,tnow,dt)

        dHpsi = dmv(y0sub)
        Hpsi = mv(y0sub)
        HdHpsi = mv(dHpsi)
        dHHpsi = dmv(Hpsi)
        Gam11 = parth1[know]*Hpsi + dt/2*dHpsi - sig*dt**2/12*(HdHpsi-dHHpsi)
        e0sub += Gam11

        y1sub, errestkry, tkrylist, mlist1 = mkprop.expimv_pKry(mv,y0sub,tol=tolkry,t=dt,m=m,ktype=2,reo=0)
        e1sub, errestkry, tkrylist, mlist2 = mkprop.expimv_pKry(mv,e0sub,tol=tolkry,t=dt,m=m,ktype=2,reo=0)
        print(know,nrm(e0sub),sum(mlist1),sum(mlist2))
        
        dHpsi = dmv(y1sub)
        Hpsi = mv(y1sub)
        HdHpsi = mv(dHpsi)
        dHHpsi = dmv(Hpsi)
        Gam12 = parth2[know]*Hpsi + dt/2*dHpsi + sig*dt**2/12*(HdHpsi-dHHpsi)
        e1sub += Gam12
        e0sub = e1sub
        y0sub = y1sub

    # computer error estimate
    y1 = y1sub
    mv2, _ = Hamiltonian.setupHamiltonian(tnow+dt)
    ye1 = mv2(y1)
    def1 = e1sub-0.5*ye1
    errestmag = dt/(p+1)*nrm(def1)
    errestlist1[ij]=errestmag
    tolkry=1e-3*errestmag
    #print(errestmag)
    
    refsteps=4
    y0sub=u
    t=tnow
    dt2=dt/refsteps
    for j in range(refsteps):
        for know in range(jexps2):
            anow = amat2[know]
            mv, dmv = Hamiltonian.setupHamiltonianCFM(anow,cmat2,chat,t,dt2)
            y1sub, errestkry, tkrylist, mlist1 = mkprop.expimv_pKry(mv,y0sub,tol=tolkry,t=dt2,m=m,ktype=2,reo=0)
            y0sub = y1sub
        t+=dt2
    testerr = nrm(y1sub-y1)
    errlist[ij]=testerr
    #print(testerr)
        
        

In [ ]:
plt.loglog(dts, errestlist1,'+-',dts,errlist,'-x',dts,1e-1*dts**(p+1),':')

In [ ]:
plt.loglog(dts, errestlist1j2,dts,errestlist1j3,dts,1e1*dts**(p+1),':')

In [ ]:
plt.loglog(dts,errlist,'-x',dts,errestlist1,'+-',dts,1e2*dts**(3),':')